<a href="https://colab.research.google.com/github/ChiranjivMalhi/Sentiment-Classifier/blob/main/tweet_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
from tqdm import tqdm

query = 'Russia Ukraine'
tweets_list = []
limit = 15000
progress_bar = tqdm(total=limit)

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets_list) >= limit:
        break
    tweets_list.append([tweet.date, tweet.user.username, tweet.rawContent])
    progress_bar.update(1)

df = pd.DataFrame(tweets_list, columns=['Date', 'User', 'Tweet'])
df.to_csv('tweets.csv', index=False)


100%|█████████▉| 14997/15000 [09:23<00:00, 23.24it/s]

In [ ]:
df = pd.read_csv('tweets.csv')

In [ ]:
df.shape

(15002, 3)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline('text-classification', model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# Testing the classifier
predicted = classifier('I am so happy today')
predicted

[[{'label': 'sadness', 'score': 0.00039981905138120055},
  {'label': 'joy', 'score': 0.9986878037452698},
  {'label': 'love', 'score': 0.0004339338920544833},
  {'label': 'anger', 'score': 0.0002096794924000278},
  {'label': 'fear', 'score': 0.00013056259194854647},
  {'label': 'surprise', 'score': 0.0001382477639708668}]]

In [ ]:
def add_emotion(df, classifier):
    emotions = []
    for tweet in df['Tweet']:
        try:
            pred = classifier(tweet)
            emotion = max(pred[0], key=lambda x:x['score'])['label']
        except:
            emotion = ''
        emotions.append(emotion)
    df['emotion'] = emotions
    return df

In [ ]:
df = add_emotion(df, classifier)
df.head()

,Date,User,Tweet,emotion
0,2023-04-17 15:38:19+00:00,TheDizzle669,@GuitarFamilyMan @brooklyn_jenny @Victorshi202...,anger
1,2023-04-17 15:38:15+00:00,MountainDogMa,@DonaldJTrumpJr Stop lying. There are only adv...,anger
2,2023-04-17 15:38:14+00:00,PopescuCo,Noooo!!! You don't say!!! What a surprise for ...,anger
3,2023-04-17 15:38:12+00:00,darwinesh,@WarMonitors This war reminds me of the war be...,joy
4,2023-04-17 15:38:10+00:00,Orhan583441,"🔥Artillery of the 6th ""Cossack Regiment"" of th...",anger


In [ ]:
df.to_csv('tweets_with_emotions.csv', index=False)